# Sequence Generation : Shakespeare Poems with RNN

In [ ]:
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.layers import Dense, Activation, Input,Dropout
from tensorflow.keras.layers import LSTM

In [3]:
text=open("shakespeare.txt",'r').read()
vocab=sorted(set(text))
char2idx={ch:i for i,ch in enumerate(vocab)}
idx2char={i:ch for i,ch in enumerate(vocab)}

In [4]:
def build_data(text,tx=40,stride=3):
    '''
    Divides input text to X and Y list
    
    Arguments:
    text -- text data (str)
    tx   -- no. of timesteps (int)
    stride--move pointer according to stride (int)
    
    Returns:
    X -- list in with each item is text of length tx
    Y -- list of expected output or each item in X
    
    '''
    X,Y=[],[]
    for i in range(0,len(text)-tx,stride):
        X.append(text[i:i+tx])
        Y.append(text[i+tx])
    
    print("Total training examples : ",len(X))
    return X,Y

In [5]:
def vectorization(X,Y,n_x,char2idx,tx=40):
    '''
    Vectorization of training data. 
    
    Arguments:
    X -- list in with each item is text of length tx
    Y -- list of expected output or each item in X
    n_x -- vocab size (int)
    char2idx -- dictionary with char:index pairs
    tx -- no. of timesteps (int)
    
    Returns:
    x -- 3D array of size (no. of training examples,tx,n_x)
    y -- 2D array of size (no. of training examples,n_x)
    
    '''
    
    m=len(X)
    x=np.zeros((m,tx,n_x))
    y=np.zeros((m,n_x))
    
    for i,sentence in enumerate(X):
        for t,char in enumerate(sentence):
            x[i,t,char2idx[char]]=1
        y[i,char2idx[Y[i]]]=1
        
    return x,y

In [6]:
def build_model(n_x,tx=40):
    '''
    Builds model
    Arguments:
    n_x -- vocab size (int)
    tx -- no. of timesteps (int)
    
    Returns:
    model -- tensorflow model
    
    '''
    inputs=Input(shape=(tx,n_x))
    #lstm_out1=LSTM(128,return_sequences=True)(inputs)
    #dropout1=Dropout(0.2)(lstm_out1)
    lstm_out2=LSTM(128)(inputs)
    dropout2=Dropout(0.2)(lstm_out2)
    outputs=Dense(n_x,activation="softmax")(dropout2)
    model=Model(inputs=inputs,outputs=outputs)
    return model

In [7]:
import sys
def generate_output(model,tx,n_x,char2idx,output_chars=100):
    '''
    Generates output 
    Arguments:
    model -- tensorflow model
    n_x -- vocab size (int)
    tx -- no. of timesteps (int)
    char2idx -- dictionary with char:index pairs
    output_chars -- no. of characters to generate (int)
    
    Returns:
    None
    
    '''
    
    generated=''
    user_input=input("Your text")
    sentence=sentence='0'*(tx-len(user_input))+user_input
    generated+=user_input

    sys.stdout.write("\n\nHere is your poem: \n\n") 
    #sys.stdout.write(usr_input)
    print(user_input,end='')
    
    for i in range(output_chars):
        x=np.zeros((1,tx,n_x))
        for t,ch in enumerate(sentence):
            if ch!='0':
                x[0,t,char2idx[ch]]=1
        
        y_pred=model(x) #returns softmax probability of every word in vocab
        #print("sum",tf.reduce_sum(y_pred))
        sampled_idx=np.random.choice(range(len(vocab)),p=np.squeeze(np.array(y_pred))) #sample by probability to generate new sequence
        nextchar=idx2char[sampled_idx]
        
        generated+=nextchar
        sentence=sentence[1:]+nextchar
        
        sys.stdout.write(nextchar)
        #sys.stdout.flush()
    #print(generated)

In [8]:
n_x=len(vocab) #vocab size
tx=40 #total timesteps
stride=3
X,Y=build_data(text,tx,stride)
x,y=vectorization(X,Y,n_x,char2idx,tx)

Total training examples :  31412


In [8]:

model=build_model(n_x,tx)
model.summary()

2023-01-21 11:36:00.219849: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-21 11:36:00.220784: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-21 11:36:00.220871: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (7655cbfd5bcd): /proc/driver/nvidia/version does not exist
2023-01-21 11:36:00.222673: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 40, 61)]          0         
                                                                 
 lstm (LSTM)                 (None, 128)               97280     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 61)                7869      
                                                                 
Total params: 105,149
Trainable params: 105,149
Non-trainable params: 0
_________________________________________________________________


In [11]:
#Saves models in every 5 epochs
class CustomSaver(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if epoch%5 == 0:  # or save after some epoch, each k-th epoch etc.
            self.model.save("models/my_shakespeare_model/my_shakespeare_model_epoch_{}.h5".format(epoch))

saver=CustomSaver()

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),metrics=['accuracy'])
history=model.fit(x,y,batch_size=64,epochs=350,callbacks=[saver])

Epoch 1/350
491/491 [==============================] - 44s 80ms/step - loss: 2.8162 - accuracy: 0.2375
Epoch 2/350
491/491 [==============================] - 41s 82ms/step - loss: 2.3965 - accuracy: 0.3242
Epoch 3/350
491/491 [==============================] - 47s 95ms/step - loss: 2.2484 - accuracy: 0.3541
Epoch 4/350
491/491 [==============================] - 47s 96ms/step - loss: 2.1528 - accuracy: 0.3777
Epoch 5/350
491/491 [==============================] - 47s 95ms/step - loss: 2.0876 - accuracy: 0.3967
Epoch 6/350
491/491 [==============================] - 47s 96ms/step - loss: 2.0341 - accuracy: 0.4087
Epoch 7/350
491/491 [==============================] - 47s 96ms/step - loss: 1.9899 - accuracy: 0.4180
Epoch 8/350
491/491 [==============================] - 48s 97ms/step - loss: 1.9505 - accuracy: 0.4273
Epoch 9/350
491/491 [==============================] - 48s 97ms/step - loss: 1.9158 - accuracy: 0.4353
Epoch 10/350
491/491 [==============================] - 48s 97ms/step - l

In [9]:
#latest saved model is model with epoch 130, so load it 
saved_model=load_model("models/my_shakespeare_model/my_shakespeare_model_epoch_130.h5")

2023-01-21 13:56:53.452220: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-21 13:56:53.453535: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-21 13:56:53.453746: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (7655cbfd5bcd): /proc/driver/nvidia/version does not exist
2023-01-21 13:56:53.456183: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [15]:
output_chars=400
generate_output(saved_model,tx,n_x,char2idx,output_chars)

Your text thou evil shall not rise 




Here is your poem: 

thou evil shall not rise incherss,
But me true by to pice a livet deaish,
And my desprade ade make If far my me pit:
And sun doth bll to sue small sight suale from cheer
Adcinlt heart's breath henounoon Maines bling meathe, tree,
As prife heaves of the wind compare chore,
My seavere on thin times of bery will:
Mo eyes sweet in in still I senter dith
To slowe, bornce the be wothing to me day
What ever surmed, huwbless not 